In [1]:
import os
import cv2
import shutil
import numpy as np
from PIL import Image
import shutil
from scipy.spatial.distance import cdist
import time
from imutils import paths
from matplotlib import pyplot as plt
import random
import math
from tqdm import tqdm


import zipfile
import os 
import cv2



import warnings
warnings.filterwarnings('ignore')
import shutil
import os
import pickle
import pandas as pd
import numpy as np
from numpy.matlib import repmat
import zipfile
from PIL import Image
from matplotlib import pyplot as plt

from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing import image

from tqdm import tqdm
from matplotlib import pyplot as plt
from PIL import Image
from imutils import paths

class Watermelon_Segmentor(object):
    def __init__(self, hsv_h, hsv_s, hsv_v):
        self.hsv_h_low = hsv_h[0]
        self.hsv_h_high = hsv_h[1]
        self.hsv_s_low = hsv_s[0]
        self.hsv_s_high = hsv_s[1]
        self.hsv_v_low = hsv_v[0]
        self.hsv_v_high = hsv_v[1]
    def rescale_and_pad_np_img_to(self,
        np_image: np.ndarray,
        target_size,
        background=(25, 55, 115)
    ) -> np.ndarray:
        w_t, h_t = target_size
        h, w, c = np_image.shape

        if w / h < w_t / h_t:
            new_w = h * w_t / h_t
            padding = new_w - w
            image = cv2.copyMakeBorder(
                np_image,
                0, 0,
                int(padding // 2), int(padding // 2),
                cv2.BORDER_CONSTANT, value=background
            )
        else:
            new_h = w * h_t / w_t
            padding = new_h - h
            image = cv2.copyMakeBorder(
                np_image,
                int(padding // 2), int(padding // 2),
                0, 0,
                cv2.BORDER_CONSTANT, value=background
            )
        return cv2.resize(image, target_size)

    def add_padding(self, img):
        ht, wd, cc= img.shape
    #     print(img.shape)

        # create new image of desired size and color (blue) for padding
        ww = int(wd + wd*0.3)

        hh = int(ht + ht*0.3)
    #     print(ww,hh)
        color = (115, 55, 25)
        result = np.full((hh,ww,cc), color, dtype=np.uint8)

        # compute center offset
        xx = (ww - wd) // 2
        yy = (hh - ht) // 2

        # copy img image into center of result image
        result[yy:yy+ht, xx:xx+wd] = img
        return result

    def segment(self, np_image):
        hsv = cv2.cvtColor(np_image, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(hsv)
        ret, thresh_h = cv2.threshold(h, self.hsv_h_low, self.hsv_h_high, cv2.THRESH_BINARY_INV)
        ret, thresh_s = cv2.threshold(s, self.hsv_s_low, self.hsv_s_high, cv2.THRESH_BINARY_INV)
        ret, thresh_v = cv2.threshold(v, self.hsv_v_low, self.hsv_v_high, cv2.THRESH_BINARY_INV)
        
        contours, hierarchy = cv2.findContours(thresh_h, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        contour_list = []
        contours = [i for i in contours if cv2.contourArea(i) > 1000]
        return contour_list, contours, [thresh_h, thresh_s, thresh_v]
    
    
    def rotate_boxes(self, img, box, center, multiplication_factor = 0.008207485226526593):
        rows, cols, _ = img.shape

        length = cdist([box[1]], [box[0]]) 
        width = cdist([box[2]], [box[1]]) 

        length_mm = max(length[0][0], width[0][0]) * multiplication_factor
        width_mm = min(length[0][0], width[0][0]) * multiplication_factor

        angle = np.arctan2((box[1][1] - box[0][1]), (box[1][0] - box[0][0])) if length > width\
        else np.arctan2((box[2][1] - box[1][1]), (box[2][0] - box[1][0])) 

        print(length[0][0], width[0][0], length[0][0] * width[0][0])

        if max(length[0][0], width[0][0]) > 0 and length[0][0] * width[0][0] < 50000000:
            rot = cv2.getRotationMatrix2D(center, math.degrees(angle) + 90, 1)
            box = np.int0(cv2.transform(np.array([box]), rot))[0]


            # Corner points
            # (x1, y1)--------------------(x2, y1)
            #    |        (corner pts)        |
            # (x2, y1)--------------------(x2, y2)
            x1 = min([box[0][0], box[1][0], box[2][0], box[3][0]])
            y1 = min([box[0][1], box[1][1], box[2][1], box[3][1]])
            x2 = max([box[0][0], box[1][0], box[2][0], box[3][0]])
            y2 = max([box[0][1], box[1][1], box[2][1], box[3][1]])

            padd = max([rows, cols]) // 50

            affine_img = cv2.warpAffine(img, rot, (rows + 1000,cols + 1000))
            minm_img = affine_img[
                        max(0, y1 - padd) : y2 + padd,
                        max(0, x1 - padd) : x2 + padd
                    ]
#             plot(minm_img)
            minm_img = minm_img.astype(np.uint8)
            # check w/h ratio
            h, w, _ = minm_img.shape
            w_h_ratio = w / h

            if not (w_h_ratio > 5 or w_h_ratio < 0.2):
                return minm_img, length_mm, width_mm, length_mm/width_mm
        else:
            return None, length_mm, width_mm, length_mm/width_mm
        
    def execute_optimised(self, img):
        """
            Parameters:
                numpy array representing the image

            Returns:
                image_list with each item in the list being a tuple of length 3
                image_list[0] contains img, length_mm, width_mm
        """
        img = self.add_padding(img)
        new_img = img.copy()
        img = cv2.resize(img, None, fx=0.2, fy=0.2)
        conts, contours, thresh = self.segment(img)
    #     print(len(contours))
        img_list = []
        box_list = []
        print(len(contours))
        for conti in contours:
            rect = cv2.minAreaRect(conti)
            box = np.int0(cv2.boxPoints(rect))
            box = box * 5
            center = tuple([i * 5 for i in rect[0]])
    #         print(box)
            minm_img, length_mm, width_mm, l_b_ratio = self.rotate_boxes(new_img, box, center)
            if minm_img is not None:
                extracted_img = self.rescale_and_pad_np_img_to(minm_img, (224, 448), (115, 55, 25))
                extracted_img = cv2.cvtColor(extracted_img, cv2.COLOR_BGR2RGB)
                img_list.append((extracted_img, length_mm, width_mm, l_b_ratio))

        return img_list
    
def RunSegmentor(input_folder_path,output_folder_path, save_type='plt', save_format='jpg'):
    cseg = Watermelon_Segmentor((75, 200), (10, 200), (10, 200))
    for index, image in enumerate(tqdm(os.listdir(f"{input_folder_path}"))):
        img = cv2.imread(f"{input_folder_path}/{image}")
        try:
            img_list = cseg.execute_optimised(img)
            if img_list is not None:
                if save_type == 'plt':
                    [plt.imsave(f"{output_folder_path}/{image}_ext{j}.{save_format}", i[0]) for j, i in enumerate(img_list)]
                elif save_type == 'cv2':
                    [cv2.imwrite(f"{output_folder_path}/{image}_ext{j}.{save_format}", cv2.cvtColor(i[0], cv2.COLOR_BGR2RGB)) for j, i in enumerate(img_list)]
                else:
                    print("Not a supported save type : Use plt or cv2 instead")
        except:
            print(f'{input_folder_path}/{image}')
    




In [2]:

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# utils path
classifier_model_path = 'smc_watermelon_v1_without_decoder.h5'

smc_classes = pickle.load(open('smc_watermelon_v1_class_names.pkl', 'rb'))['class_names']
print('for classification > {}, classes {}'.format(classifier_model_path, smc_classes))

for classification > smc_watermelon_v1_without_decoder.h5, classes ['bad', 'good']


In [3]:
#@title smc class

import tensorflow as tf
def my_init(shape, dtype=None):
    initializer = tf.keras.initializers.he_uniform(seed = 1)
    return initializer(shape, dtype=dtype)

def _no_loss(y_true, y_pred):
    '''
    loss function is not required for inference
    '''
    return np.float32(0)

classifier = load_model(classifier_model_path,  compile=False)

# classifier.compile(
#     optimizer=SGD(),
#     loss={
#         'decoder_o/p':_no_loss, # (uncomment only if AEC model)
#         'classifier_o/p': _no_loss
#     }
# )

def show(objs, titles, structure='LR', figsize=(10, 5), clims=False, axes=False, xlabels=False):
    '''
    utility function to show plots (subplots)
    '''
    plt.style.use('default')
    plt.figure(figsize=figsize)
    for i, (obj, name) in enumerate(zip(objs, titles)):
        if structure == 'LR': plt.subplot(1, len(titles), i+1)
        elif structure == 'TB': plt.subplot(len(titles), 1, i+1)
        plt.imshow(obj)
        if axes: plt.axis(axes[i])
        else: plt.axis('off')
        if xlabels: plt.xlabel(xlabels[i])
        if clims: plt.clim(clims[i])
        plt.title(name)
    plt.show()

def predict_seed_types(im_path, dsize=(448, 224)):
    '''
    predict from path.. (later)
    '''
    img = image.load_img(im_path, color_mode='rgb', target_size=dsize)
    img = image.img_to_array(img).astype('float32') / 255.0

    y = classifier.predict(np.expand_dims(img, axis=0))
    y = smc_classes[np.argmax(y)]
    return y
    

In [ ]:
#cd anand/seed
#aws s3 cp s3://seedvision-images/paddy/foundation_seed/4-8-2021/raw_images/055.zip .  

In [ ]:
!unzip ../seed/055.zip -d .

In [ ]:
!unzip "../seed/055.zip" -d "../seed/"

In [ ]:
with zipfile.ZipFile('anand/seed/055.zip', 'r') as zip_ref:
    zip_ref.extractall('anand/seed/')

In [ ]:
!ls

In [22]:
inp_ex ="6p4"
out_ex='083_ex'
inp_smc = out_ex
out_smc='083_smc'

In [23]:
list = os.listdir(inp_ex)
len(list)

4195

In [24]:
RunSegmentor(inp_ex,out_ex, save_type='cv2', save_format='jpg')

  0%|          | 1/4195 [00:00<26:48,  2.61it/s]

1
1094.8972554536795 531.8364410229897 582306.259626324


  0%|          | 2/4195 [00:00<23:59,  2.91it/s]

0


  0%|          | 3/4195 [00:01<25:26,  2.75it/s]

...

0


100%|█████████▉| 4194/4195 [20:00<00:00,  4.19it/s]

0


100%|██████████| 4195/4195 [20:01<00:00,  3.49it/s]

0


In [25]:
list = os.listdir(out_ex)
len(list)

3966

In [26]:
path = inp_smc
new_path = out_smc

In [27]:

img_paths = []
main_types = []
class_labels = []

IGNORE = []

# for class_ in sorted(os.listdir(path)):
#     imgDir = '{}{}'.format(path, class_)
#     print('* directory: {}'.format(imgDir))

for imgFname in sorted(os.listdir(path)):
    if imgFname.endswith('.jpg'):
        #img_paths.append('{}/{}'.format(imgDir, imgFname))
        img_paths.append(os.path.join(path,imgFname))
        #class_labels.append(class_)
            
print('Total samples: {}'.format(len(img_paths)))

data = pd.DataFrame()
data['img_path'] = img_paths
# data['class_label'] = class_labels
data = data.sample(frac=1.0, random_state=0)
data.tail()

Total samples: 3966


,img_path
835,083_ex/img_1630492726605.jpg_ext0.jpg
3264,083_ex/img_1630494054307.jpg_ext0.jpg
1653,083_ex/img_1630493186178.jpg_ext1.jpg
2607,083_ex/img_1630493732558.jpg_ext1.jpg
2732,083_ex/img_1630493785540.jpg_ext0.jpg


In [28]:

from shutil import copyfile, rmtree


dir_ = os.path.join(new_path,"good") #path.replace('extracted', 'extracted_good_seeds')
for clas in smc_classes:
  # other_dir = path.replace('extracted', 'extracted_other_seeds')
  #os.mkdir(clas)
  other_dir = os.path.join(new_path,"bad")



try:
  rmtree(dir_)
  rmtree(other_dir)
except: pass
os.mkdir(dir_)

os.mkdir(other_dir)
[os.mkdir('{}/{}'.format(other_dir, class_)) for class_ in smc_classes]

[None, None]

In [29]:


%%time

num_good_seeds = 0

for i, im_path in enumerate(data['img_path']):
  smc_label = predict_seed_types(im_path)
  if smc_label == 'good_seed':
      num_good_seeds += 1
      print('{}/{} (Total good seeds: {}) Processing {}---------|'
            .format(i+1, len(data), num_good_seeds,im_path), end='\r')
      
      # save a copy to new dir_
      copyfile(im_path, im_path.replace(path, dir_))
  else:
      copyfile(im_path, '{}/{}/{}'.format(other_dir, smc_label, im_path.split('/')[-1]))

CPU times: user 2min 23s, sys: 9.27 s, total: 2min 32s
Wall time: 2min 30s


In [21]:
7list = os.listdir(os.path.join(out_smc , 'good'))
len(list)

0

In [ ]:
shutil.make_archive(out_ex,'zip',out_ex)

In [ ]:
shutil.make_archive(out_smc,'zip',out_smc)